In [13]:
import pandas as pd
import requests
import time
import websocket
import json, pprint, talib 
import numpy
from binance.client import Client
from binance.enums import *
from coinmarketcapapi import CoinMarketCapAPI, CoinMarketCapAPIError

cmc = CoinMarketCapAPI('a01ace6b-f43c-439e-9fca-dc7f73e2a648')


binance_api_key = 'ihvXWbHvvDrYuroIJOS9ANW6X7tv5OQSYibLLloKyIGui0w93Q5yUJOBZclnNL5D'
binance_api_secret = 'uzmKBJ3lmVS4TsTl2AOHVJOYTFfLM5qrg00vAqUyTYbSjiy7BDPlTpaq2Oy7qdo2'
client = Client(binance_api_key, binance_api_secret)


def get_all_tokens():
    #filter out leverage tokens, non usd tokens, 
    all_pairs = pd.DataFrame(client.get_ticker())
    relev = all_pairs[all_pairs.symbol.str.endswith('USDT')]
    non_lev_list = relev[ ~( (relev.symbol.str.contains('UP')) | (relev.symbol.str.contains('DOWN')) | (relev.symbol.str.contains('BEAR')) | (relev.symbol.str.contains('BULL')) ) ]
    #filter out stable coins
    non_stable_rm = non_lev_list[~all_pairs.symbol.str.startswith('USD')]
    #remove BCHSV, PAX, STRAT, XZC
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'BCHSVUSDT'].index )
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'PAXUSDT'].index ) # dollar value
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'STRATUSDT'].index )
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'XZCUSDT'].index )
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'AUDUSDT'].index ) # non crypto
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'EURUSDT'].index ) # non crypto
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'GBPUSDT'].index ) # non crypto
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'BCCUSDT'].index ) # BitConnect scam
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'BCHABCUSDT'].index ) # Bitcoin Cash ABC (BCHA) Rebrand to eCash (XEC)
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'BUSDUSDT'].index ) # Binance USD
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'BTGUSDT'].index ) # Bitcoin Gold
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'ERDUSDT'].index ) # Elrond (ERD) Mainnet & Token Swap to Elrond Gold (EGLD) 
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'LENDUSDT'].index ) # LEND Token Swap to AAVE
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'MCOUSDT'].index ) # MCO Token unsupported

    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'GXSUSDT'].index ) # Find a way to replace lookup
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'IOTAUSDT'].index ) # Find a way to replace lookup

    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'NPXSUSDT'].index ) # Pundi X old -> PUNDIX
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'PAXGUSDT'].index ) # Pax Gold Binance Completes VEN Mainnet Swap to VET
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'VENUSDT'].index ) # Binance Completes VEN Mainnet Swap to VET 
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'TUSDUSDT'].index ) # stable coin TrueUSD 
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'STORMUSDT'].index ) #Storm (STORM) Token Swap to STMX  NPXS	 
    non_stable_rm = non_stable_rm.drop( non_stable_rm[non_stable_rm.symbol == 'SUSDUSDT'].index ) # usd coin sUSD
    #sorted = non_stable_rm.sort_values(by='priceChangePercent', ascending=False)
    sorted = non_stable_rm.sort_values(by='symbol', key=lambda x: x.str.split('USDT').str[0])

    token_tickers = sorted
    symbols=token_tickers['symbol'].str.slice(0,-4)
    symbols = symbols.replace('GXS', 'GXC')
    symbols = symbols.replace('IOTA', 'MIOTA')
    symbolslist = ",".join(symbols)

    cmcinfo = cmc.cryptocurrency_info(symbol=symbolslist)
    names_info = cmcinfo.data
    names = []
    for ind in names_info:
        names.append(names_info[ind]['name'].replace('[', '').replace(']', '').replace('(', '') .replace(')', ''))
        
    sorted.insert(1, "name", names)
    #sorted['names'] = names

    return sorted

token_tickers = get_all_tokens()

C:\Users\Public\Documents\Wondershare\CreatorTemp/ipykernel_9892/4010754561.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  non_stable_rm = non_lev_list[~all_pairs.symbol.str.startswith('USD')]


In [14]:
symbols=token_tickers['symbol'].str.slice(0,-4)
symbolslist = ",".join(list(symbols))
cmcinfo = cmc.cryptocurrency_info(symbol=symbolslist)

names_info = cmcinfo.data
names = []
for ind in names_info:
    names.append(names_info[ind]['name'])




name = pd.Series(names)
ticker = pd.Series(list(symbols))
row_df = pd.DataFrame()
row_df['ticker'] = ticker
row_df['name'] = name
pd.set_option('display.max_rows', 400)
row_df


,ticker,name
0,1INCH,1inch Network
1,AAVE,Aave
2,ACM,AC Milan Fan Token
3,ADA,Cardano
4,ADX,Ambire AdEx
5,AGLD,Adventure Gold
6,AION,Aion
7,AKRO,Akropolis
8,ALGO,Algorand
9,ALICE,MyNeighborAlice


In [12]:
cmcinfo = cmc.cryptocurrency_info(symbol='MIOTA')
names_info = cmcinfo.data

names = []
for ind in names_info:
    names.append(names_info[ind]['name'])

print(names)


['IOTA']
